In [ ]:
import psycopg2 as db
from uuid import uuid4
import hashlib
import importnb

with importnb.Notebook():
    import rag

def get_connection():
    conn = db.connect(
        host="localhost",
        database="metadata",
        user="postgres",
        password="khanzode"
    )
    print("Connection established")
    return conn

conn = get_connection()
cur = conn.cursor()

Connection established
Connected to the database successfully
Database initialized successfully
Connection established
Connected to the convo database successfully


[nltk_data] Error loading averaged_perceptron_tagger_eng: <urlopen
[nltk_data]     error [SSL: CERTIFICATE_VERIFY_FAILED] certificate
[nltk_data]     verify failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1028)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>


In [ ]:
def init_db():
    cur.execute("""
        CREATE TABLE IF NOT EXISTS query_feedback_logs (
            id SERIAL PRIMARY KEY,
            query TEXT,
            response TEXT,
            response_id TEXT,
            feedback TEXT DEFAULT 'None',
            bad_feedback_reason TEXT DEFAULT 'None',
            session_id TEXT,
            query_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    print("Convo table initialized successfully")

In [ ]:
def add_entry(entry):
    print("-"*100)
    cur.execute("""
        INSERT INTO query_feedback_logs (query, response, response_id, session_id)
        VALUES (%s, %s, %s, %s)
        RETURNING id
    """, (entry["query"], entry["response"], entry["response_id"], entry["session_id"]))
    
    query_id = cur.fetchone()[0]
    conn.commit()
    print(f"Inserted query-response in db with ID: {query_id}")
    print("-"*100)
    return query_id

def update_feedback(response_id, thumb_sign, reason):
    print("-"*100)
    if thumb_sign == "down":
        feedback = "Bad"
    elif thumb_sign == "up":
        feedback = "Good"
    else:
        feedback = "None"
    
    cur.execute("""
        UPDATE query_feedback_logs 
        SET feedback = %s, bad_feedback_reason = %s
        WHERE response_id = %s
    """, (feedback, reason, response_id))
    
    conn.commit()
    print(f"Feedback updated for query ID: {response_id}")
    print("-"*100)
    print(f"Feedback updated for query ID: {response_id}")
    print("-"*100)


In [ ]:
def get_bad_query_response():
    cur.execute("SELECT id, query, response, response_id, feedback, session_id, query_timestamp FROM query_feedback_logs WHERE feedback = 'Bad'")
    bad_qrs = cur.fetchall()
    
    # Convert tuples to dictionaries for easier access
    result = []
    for row in bad_qrs:
        result.append({
            "id": row[0],
            "query": row[1],
            "response": row[2],
            "response_id": row[3],
            "feedback": row[4],
            "session_id": row[5],
            "query_timestamp": row[6]
        })
    
    return result

In [ ]:
def get_thumbs_down_query(response_id):
    cur.execute("SELECT query, response, bad_feedback_reason from query_feedback_logs WHERE response_id = %s", (response_id,))
    return cur.fetchone()